In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime, timedelta

# Indicator Creation

In this notebook we will create new interesting features which we believe to be relevant for clustering purposes. The indicators we will try to create in this notebook are:
1. Account age in days (From account creation till the release of this dataset) (datetime(2022,9,29,11,0,0))
2. Account average tweets per day
3. Account highest daily tweet count
4. Account average twitter text length
5. Account tweet hashtag average
6. Account discussion creation score (Tweet to retweet ratio)
7. Account average number of mentions per tweet

In [2]:
merged_df = pd.read_csv("dataset/merged_dataset.csv")
merged_df.head(1)

,Unnamed: 0,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text
0,0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu..."


In [3]:
merged_df.axes
merged_df.drop(columns="Unnamed: 0", inplace=True) # Dropping the first column of the dataset, consisting of an index copy
merged_df.axes

[RangeIndex(start=0, stop=10599305, step=1),
 Index(['Unnamed: 0', 'user_id', 'name', 'lang', 'bot', 'account_created',
        'statuses_count', 'tweet_id', 'retweet_count', 'reply_count',
        'favorite_count', 'num_hashtags', 'num_urls', 'num_mentions',
        'tweet_created', 'text'],
       dtype='object')]

[RangeIndex(start=0, stop=10599305, step=1),
 Index(['user_id', 'name', 'lang', 'bot', 'account_created', 'statuses_count',
        'tweet_id', 'retweet_count', 'reply_count', 'favorite_count',
        'num_hashtags', 'num_urls', 'num_mentions', 'tweet_created', 'text'],
       dtype='object')]

## 1. Account age in days
Creating the account age in days feature and adding it to the dataframe.

In [4]:
merged_df["account_created"] = pd.to_datetime(merged_df["account_created"], errors="coerce") # Even though this is already done in the previous notebook, this information is turned to string when converting the merged_df to csv

release_of_dataset = datetime(2022,9,29,11,0,0)
account_age_in_days = []

for elem in merged_df["account_created"]:
    age = release_of_dataset - elem
    age_in_days = age.days
    account_age_in_days.append(age_in_days)

merged_df["account_age_in_days"] = account_age_in_days
merged_df.head()

,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text,account_age_in_days
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu...",1314
1,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.795354e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-06-21 08:05:13,"""Happiness is not a station you arrive at, but...",1314
2,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.517009e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-05 12:41:00,Music flow.,1314
3,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.919200e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-25 12:17:31,@_SimplyKC follow meeee...,1314
4,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.605225e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-29 20:55:05,"""You are the only person on earth who can use ...",1314


# 2. Account average tweets per day

Creating the account average tweets per day and adding it to the dataframe.

In [16]:
list_of_average_tweets_per_day = []

for elem in merged_df["statuses_count"]:
    try:
        tweets_made = elem
        age_in_days = merged_df["account_age_in_days"][elem]
        average_tweets_per_day = int(tweets_made)/int(age_in_days)
        list_of_average_tweets_per_day.append(average_tweets_per_day)
    except KeyError:
        list_of_average_tweets_per_day.append(np.nan)

merged_df["account_average_tweets_per_day"] = list_of_average_tweets_per_day
merged_df.head()

,user_id,name,lang,bot,account_created,statuses_count,tweet_id,retweet_count,reply_count,favorite_count,num_hashtags,num_urls,num_mentions,tweet_created,text,account_age_in_days,account_average_tweets_per_day
0,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.867187e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-11 03:49:06,"@4fri2endly0 ""A business that makes nothing bu...",1314,0.057839
1,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.795354e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-06-21 08:05:13,"""Happiness is not a station you arrive at, but...",1314,0.057839
2,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.517009e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-05 12:41:00,Music flow.,1314,0.057839
3,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.919200e+17,0.0,0.0,0.0,0.0,0.0,1.0,2019-07-25 12:17:31,@_SimplyKC follow meeee...,1314,0.057839
4,2353593986,Lamonica Raborn,en,1,2019-02-22 18:00:42,76.0,4.605225e+17,0.0,0.0,0.0,0.0,0.0,0.0,2019-04-29 20:55:05,"""You are the only person on earth who can use ...",1314,0.057839
